In [ ]:
##########################Load External Libraries  ####################################
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, metrics
from ipywidgets import widgets, interactive
import gc
import joblib
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta 
from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm
from itertools import cycle
import datetime as dt
from torch.autograd import Variable
import random 
from matplotlib.pyplot import figure
from fastprogress import master_bar, progress_bar
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import time 
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import os
from datetime import datetime, time
import torch.nn.functional as F
from sklearn.svm import SVR
import optuna
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import onnx
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
from Module.feat_engineer import time_ratio, sliding_windows
from Module.data_clean import data_filter_svr, plot_data_before_after
from Module.Time_alignment import shift_and_remove_bottom_rows, slide_y, slide_y_pre

In [ ]:
from Model.CNN_BiLSTM import CNNLSTMModel, init_weights
from Model.CNN_Transformer import CustomTransformer

In [ ]:
device = 'cuda:0'
path = r"data/"

In [ ]:
# 1day
window_size = 96 

In [ ]:
# read data
input, output = [], []
station = []
xlist, ylist = [], []
input.append( path + 'A.csv')
output.append( path + '2023-measured-A.csv')

# Data regularization
for i in range(1):
    I = pd.read_csv(input[0]) # changeable
    # I = I.to_frame()
    I.set_index(I.columns[0], inplace=True)
    I.interpolate(method='linear', axis=0, inplace=True)
    
    O = pd.read_csv(output[0]).iloc[:, [1, 2]]
    O.set_index(O.columns[0], inplace=True)
    O.interpolate(method='linear', axis=0, inplace=True)
    p = I.join(O, how='outer') # Merge by date
    print(I.shape,O.shape)
    
    # Move the data in the last column down the window_size row
    p.iloc[:, -1] = p.iloc[:, -1].shift(-window_size)
    p.dropna(axis=0, inplace = True)

    # data clean
    p_before = p
    p = data_filter_svr(p)
    p_after = p

    # Min-Max processing
    original_max, original_min = p.iloc[:, -1].max(), p.iloc[:, -1].min() # original range recorded
    scaler = MinMaxScaler()
    p_ = pd.DataFrame(scaler.fit_transform(p), columns=p.columns)

    # time ratio column constructed
    time_index = p.index.to_numpy()
    p_['time'] = time_ratio(time_index)
    station.append((p_, time_index))

In [ ]:
for i in range(1):
    x, y = sliding_windows(station[i][0], station[i][1], window_size)
    print(i,':','x---',x.shape,'---','y---',y.shape)
    xlist.append(x)
    ylist.append(y)

In [ ]:
x1 = x[:, :, [0, 1, 2, 6]] # model-prediction
x2 = x[:, :, [3, 4, 5, 6]] # station-measured (without time_ratio)

In [ ]:
train_size = int(len(y) * 0.7)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x1)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x1[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x1[train_size:len(x1)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [ ]:
print("train shape is:",trainX.size())
print("train label shape is:",trainY.size())
print("test shape is:",testX.size())
print("test label shape is:",testY.size())

## Short-term Prediction (CNN-LSTM)

In [ ]:
# CNNLSTM
num_epochs = 2000 
input_dims = 4
lstm_units = 38
learning_rate = 0.04 
momentum=0.88# 0.9 # SGDM, empirical 
weight_decay=1e-3

In [ ]:
lstm = CNNLSTMModel(input_dims, lstm_units)
lstm.to(device)
lstm.apply(init_weights)

criterion = torch.nn.L1Loss().to(device) # MAE
optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
# When there is no improvement within a certain number of epochs, the learning rate will be automatically reduced.
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,  patience=100, factor =0.1 ,min_lr=1e-7, eps=1e-08) 


# Train CNN-LSTM MODEL
epoch_list = []
train_loss_list = []
valid_loss_list = []

for epoch in progress_bar(range(num_epochs)):
    lstm.train()
    outputs = lstm(trainX.to(device))
    optimizer.zero_grad()
    torch.nn.utils.clip_grad_norm_(lstm.parameters(), 1)
    # loss function
    loss = criterion(outputs, trainY.to(device))
    loss.backward()
    scheduler.step(loss)
    optimizer.step()
    lstm.eval()
    valid = lstm(testX.to(device))
    vall_loss = criterion(valid, testY.to(device))
    scheduler.step(vall_loss)
    
    # loss value
    epoch_list.append(epoch + 1)
    train_loss_list.append(loss.cpu().item())
    valid_loss_list.append(vall_loss.cpu().item())
    
    if (epoch + 1) % 50 == 0:
        print("Epoch: %d, loss: %1.5f valid loss:  %1.5f " %(epoch + 1, loss.cpu().item(), vall_loss.cpu().item()))

# plot
plt.figure()
plt.plot(epoch_list, train_loss_list, label='Train Loss')
plt.plot(epoch_list, valid_loss_list, label='Valid Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
torch.save(lstm, 'y_for_AT.pth') # Intermediate power prediction (Mode training result)

In [ ]:
def predict_module(lstm, dataX =  dataX, testX = testX):
    
    # employing the trained model to predict
    lstm.eval()
    train_predict = lstm(dataX.to(device))
    data_pre = train_predict.cpu().data.numpy()
    dataY_pl = dataY.data.numpy()

    # denormalization
    data_predict  = data_pre * (original_max - original_min) + original_min
    dataY_plot = dataY_pl * (original_max - original_min) + original_min

    # predicted output
    print(data_predict.shape)
    df_predict = pd.DataFrame(data_predict)

    # actual output
    df_labels = pd.DataFrame(dataY_plot)

    # plot（Entire）
    figure(num=None, figsize=(19, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(df_predict[0])
    plt.legend(['Prediction','Time Series'],fontsize = 21)
    plt.suptitle('Time-Series Prediction Entire Set',fontsize = 23)
    plt.xticks(fontsize=21 )
    plt.yticks(fontsize=21 )
    plt.ylabel(ylabel = 'Photovoltaic Power Generation',fontsize = 21)
    plt.show()

    # plot（Test）
    figure(num=None, figsize=(23, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(df_labels.iloc[-testX.size()[0]:][0].to_numpy())
    plt.plot(df_predict.iloc[-testX.size()[0]:][0].to_numpy())
    plt.legend(['Actual Output','Predicted Output'],fontsize = 21)
    plt.suptitle('CNN+BiLSTM(model only)',fontsize = 23)
    plt.xticks(fontsize=21 )
    plt.yticks(fontsize=21 )
    plt.xlabel(xlabel = 'Test Set Data Points',fontsize = 21)
    plt.ylabel(ylabel = 'Ouput',fontsize = 21)
    plt.savefig('short_north_fzbl.png',dpi=1200)
    plt.show()

    # test shape
    print('testX:', testX.size())
    # RMSE
    print('RMSE:', np.sqrt(mean_squared_error(dataY_plot[-testX.size()[0]:],data_predict[-testX.size()[0]:])))
    # MAE
    print('MAE:', mean_absolute_error(dataY_plot[-testX.size()[0]:],data_predict[-testX.size()[0]:]))
    return dataY_plot, data_predict

In [ ]:
# print out 'y_for_AT'
fusion = torch.load('y_for_AT.pth')
print('\n'+'-'*10+'Intermediate power prediction (Model prediction)'+'-'*10+'\n')
actual, y_for_AT = predict_module(fusion)
y_for_AT = list((y_for_AT - original_min)/(original_max - original_min))

In [ ]:
x2 = x2[window_size:-window_size]

In [ ]:
Y = slide_y(y_for_AT, window_size)
y_pre = slide_y_pre(y, window_size)
print(Y.shape,y_pre.shape,x2.shape)
x2= np.concatenate((x2, y_pre, Y), axis=2)

In [ ]:
y = y[window_size:-window_size]

## Multi-head attention cross-attention
[allowing historical data (q, k) to play a correction role]

In [ ]:
input_dim = 2 
d_model = 64   
num_heads = 32   
d_ff = 256    
max_len = 50000  
num_layers = 2  
output_dim = 1  
seq_len = 16 # 24【6h】
num_epochs = 500
learning_rate = 0.001 # 1e-3
batch_size = 64
weight_decay = 0.0001 #1e-4
train_losses = []
valid_losses = []
test_losses = []

In [ ]:
# train-test split for station data
train_size -= window_size
x2 = shift_and_remove_bottom_rows(x2, seq_len) # adjust to output in the past
staX = Variable(torch.Tensor(np.array(x2))).to(device)
sta_trainX = Variable(torch.Tensor(np.array(x2[0:train_size]))).to(device)
sta_testX = Variable(torch.Tensor(np.array(x2[train_size:len(x2)]))).to(device)
print('staX',staX.size())
print('sta_trainX',sta_trainX.size())
print('sta_testX',sta_testX.size())

y = y[:-seq_len]
trainY = Variable(torch.Tensor(np.array(y[0:train_size]))).to(device)
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)]))).to(device)
print(testY.shape,trainY.shape)

In [ ]:
# train set
x = sta_trainX[seq_len:-seq_len,-seq_len:,[0,1,2,3,4,-1]].to(device)  # shape (batch_size, seq_len, input_dim) 
y = slide_y_pre(trainY.to('cpu'),seq_len)      # shape (batch_size,seq_len)
y = Variable(torch.Tensor(y.squeeze(-1))).to(device)
print(x.shape,y.shape)

# test set
test_x = sta_testX[seq_len:-seq_len,-seq_len:,[0,1,2,3,4,-1]].to(device)  # 形状为 (batch_size, seq_len, input_dim)
test_y1 =slide_y_pre(testY.to('cpu'),seq_len)      # shape: (batch_size,seq_len)
test_y1 = Variable(torch.Tensor(test_y1.squeeze(-1))).to(device)
print(test_x.shape,test_y1.shape)

In [ ]:
model = CustomTransformer(d_model, num_heads, d_ff, seq_len)
model.to(device)
criterion = nn.L1Loss().to(device)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.88, weight_decay=1e-3)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=100, verbose=True)

In [ ]:
# Dataloaders
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# early stopping params
patience = 10  # early stopping patience
min_epochs_to_trigger = 200  # the minimum number of epochs before early stopping can be triggered
best_loss = np.inf
stable_epochs = 0
threshold = 0.005 # loss threshold for early stopping

# Train
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    
    for batch_x, batch_y in dataloader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        # forward
        output = model(batch_x)
        # loss calculation
        loss = criterion(output, batch_y)
        epoch_loss += loss.item()
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # learning rate 
    scheduler.step(epoch_loss)
    train_losses.append(epoch_loss / len(dataloader))
    
    # test loss calculated
    model.eval()
    with torch.no_grad():
        test_x = test_x.to(device)
        test_y1 = test_y1.to(device)
        test_output = model(test_x)
        test_loss = criterion(test_output, test_y1)
        test_losses.append(test_loss.item())
    
    # check early-stopping condition
    if epoch >= min_epochs_to_trigger:
        if abs(test_loss - best_loss) < threshold:
            stable_epochs += 1
        else:
            stable_epochs = 0

        best_loss = min(best_loss, test_loss)

        if stable_epochs >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1} due to stable loss")
            break
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}')

# save the model
torch.save(model.state_dict(), 'CA.pth')

# plot loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
if valid_losses:
    plt.plot(valid_losses, label='Validation Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training, Validation, and Test Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
KEY = -1 # 8
# load the model
CustomTransformer(d_model, num_heads, d_ff, seq_len)
model.load_state_dict(torch.load('CA.pth'))
model.eval()

# predict
with torch.no_grad():
    test_output1 = model(test_x)
    test_loss = criterion(test_output1, test_y1)
    valid_losses.append(test_loss.item())
    print(f'Test Loss: {test_loss.item():.4f}')

# denormalization
test_output = test_output1 * (original_max - original_min) + original_min
test_y = test_y1 * (original_max - original_min) + original_min

test_output = test_output.to('cpu')[:,KEY]
test_y = test_y.to('cpu')[:,KEY]
print(test_output.shape,test_y.shape)

# calculate RMSE and MAE
rmse = np.sqrt(mean_squared_error(test_y, test_output))
mae = mean_absolute_error(test_y, test_output)
print(f'Test RMSE: {rmse:.4f}')
print(f'Test MAE: {mae:.4f}')

# compare predicted and actual values
figure(num=None, figsize=(23, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(test_y.numpy(), label='True Values')
plt.plot(test_output.numpy(), label='Predicted Values')
plt.xticks(fontsize=21 )
plt.yticks(fontsize=21 )
plt.xlabel('Test Set Data Points',fontsize = 21)
plt.ylabel('Output',fontsize=21)
plt.title('CNN+Transformer(4h)',fontsize=21)
plt.legend(['Actual Output','Predicted Output'],fontsize = 21)
plt.savefig('CNN+Transformer(4h).png',dpi=1200)
plt.show()